<a href="https://colab.research.google.com/github/vidyashankarappa/DeepLearning/blob/main/DeepLearningBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install evaluate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from copy import deepcopy
import torch
import torch.nn as nn
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


In [ ]:
# Make sure to change runtime to GPU
# Check if GPU is avaiable
device = "cuda" if torch.cuda.is_available() \
else "mps" if torch.mps.is_available() \
else "cpu"
print("Device:", device)


Device: cuda


In [ ]:
!unzip IMDB\ Dataset.csv.zip

Archive:  IMDB Dataset.csv.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# split train datainto train and validation
# stratify_by_column='label' makes split equabel for classes
# Load the dataset. Replace 'your_dataset_name' with the actual name of your dataset
from datasets import load_dataset
dataset = load_dataset('imdb')
train_val_dataset = dataset['train'].train_test_split(test_size=0.2,
stratify_by_column='label')
train_dataset = train_val_dataset['train']
val_dataset = train_val_dataset['test']
# keep the original test data
test_dataset = dataset['test']
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [ ]:
# split train datainto train and validation
# stratify_by_column='label' makes split equabel for classes
train_val_dataset = dataset['train'].train_test_split(test_size=0.2,
stratify_by_column='label')
train_dataset = train_val_dataset['train']
val_dataset = train_val_dataset['test']
# keep the original test data
test_dataset = dataset['test']
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [ ]:
dataset['train'].shuffle()[:3]

{'text': ['How they got Al Pacino to play in this movie is beyond me. This movie is absolutely terrible. I discovered, after reading some of the other reviews, that a couple of people actually enjoyed this film, which deeply puzzles me, because I do not see how anyone in their right mind could possibly enjoy a movie as awful as Revolution. It\'s not just that it\'s a bad movie, with a lame plot and overall strangeness that is extremely unpleasant, but it seems as if the filmmakers were either mentally retarded (which is a very possible explanation as to why this movie sucks like it does, though it probably still sucks even compared to other films made by retards) or deliberately made every illogical decision to make this movie suck as much as possible. For example, we see Donald Sutherland running around with a huge, fat ugly mole on his face. He does not normally have a mole. The mole does not add to his character. It is extremely ugly and distracting. It\'s not like Robert De Niro\'s

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case= True)

In [ ]:
# helper function for tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
# tokenize text to numerical ids
train_dataset_tokenized = train_dataset.map(tokenize_function,
batched=True)
val_dataset_tokenized = val_dataset.map(tokenize_function, batched=True)
test_dataset_tokenized = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# handle mini-batch loading and padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
num_labels=2,
id2label=id2label,
label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# set the accuracy metric to evalate the model
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
# Define your training hyperparameters in TrainingArguments
training_args = TrainingArguments(
  output_dir='./results',   # output directory
  learning_rate=2e-5, # set learning rate
  num_train_epochs=2, # total number of training epochs
  per_device_train_batch_size=16, # batch size per device during training
  per_device_eval_batch_size=64, # batch size per device duirng evaluation
  eval_strategy="epoch", # evaluate per epoch
  save_strategy="epoch", # save the best model according to epochs
  load_best_model_at_end=True, # load the best model
  report_to='none' # disable wandb logging
)


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset_tokenized,
    eval_dataset = val_dataset_tokenized,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# Load metric functions
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
# Initialize test_results as an empty dictionary
test_results = {}
# Get model predictions
predictions = trainer.predict(test_dataset_tokenized)
logits, labels = predictions.predictions, predictions.label_ids
preds = np.argmax(logits, axis=-1)
# Compute extra metrics
accuracy = accuracy_metric.compute(predictions=preds, references=labels)
precision = precision_metric.compute(predictions=preds, references=labels,average="macro")
recall = recall_metric.compute(predictions=preds, references=labels,average="macro")
f1 = f1_metric.compute(predictions=preds, references=labels,average="macro")
# Combine results
test_results.update({
  "test_accuracy": accuracy["accuracy"],
  "test_f1": f1["f1"],
  "test_precision": precision["precision"],
  "test_recall": recall["recall"]
})
print(test_results)

{'test_accuracy': 0.48624, 'test_f1': 0.4532153105761557, 'test_precision': 0.481856733612563, 'test_recall': 0.48624}
